In computing, images are represented by arrays of pixels, where each pixel contains information about colour and brightness. We can manipulate images at the pixel level using libraries such as NumPy. Let's start by importing the libraries we require.

In [ ]:
from PIL import Image  # importing the Image module from the Python Imaging Library
import numpy as np

We can open images using the Python Imaging Library (PIL). Red, green, and blue are the primary colours, and a combination of these is enough to generate any colour.

In [ ]:
image = Image.open('tractor_img.jpg')
image

We can now convert the image to an array, which will allow us to work directly with its pixel values

In [ ]:
image_arr = np.array(image)

In [ ]:
image_arr

Let's check out the shape of this array

In [ ]:
image_arr.shape  # height x width x channels

The array is a 3D array with the dimensions (height, width, 3), where each pixel is represented by a set of three values corresponding to the RGB channels. Let's perform some simple image manipulations on our image.

### Inversion

RGB values take the range $[0, 255]$ for each colour. (0, 0, 0) for (R, G, B) corresponds to black, or the complete absence of colour, while (255, 255, 255) corresponds to white. To invert an image, therefore, we need to subtract each pixel value from 255. Thus, the inversion of white would be black while the inversion of black would be white.

In [ ]:
inverted_image_arr = 255 - image_arr

We can now convert the array back to an image and view it to verify our result

In [ ]:
inverted_image = Image.fromarray(inverted_image_arr)
inverted_image

### Convert to greyscale

To convert an image to greyscale, we need to apply a formula which will weigh each channel to arrive at the range of colours that the human eye approximately interprets as grey. The formula is as follows:

$$grey = 0.2989 \cdot R + 0.5870 \cdot G + 0.1140 \cdot B$$

In [ ]:
greyscale_image_arr = 0.2989 * image_arr[:, :, 0] + 0.5870 * image_arr[:, :, 1] + 0.1140 * image_arr[:, :, 2]

greyscale_image_arr = greyscale_image_arr.astype(np.uint8)  # as Image.fromarray() cannot handle float values

greyscale_image = Image.fromarray(greyscale_image_arr)
greyscale_image

### Adjusting brightness
Brightness can be adjusted by simply adding more of the colour value to the image. We need to ensure that the range of values does not leave $[0, 255]$; to do this, we can use the `np.clip()` function, which clips values based on lower and upper bounds.

In [ ]:
brightness_factor = 1.5  # increasing brightness by 50%
brightened_image_arr = np.clip(image_arr * brightness_factor, 0, 255).astype(np.uint8)

brightened_image = Image.fromarray(brightened_image_arr)
brightened_image

In [ ]:
brightness_factor = 0.5  # decreasing brightness by 50%
darkened_image_arr = np.clip(image_arr * brightness_factor, 0, 255).astype(np.uint8)

darkened_image = Image.fromarray(darkened_image_arr)
darkened_image

### Adjusting contrast

Contrast can be adjusted by scaling pixel values around the mean image brightness. Increasing the distance from the mean for each pixel boosts contrast.

In [ ]:
mean = np.mean(image_arr)  # find the mean image brightness
contrast_factor = 0.5  # increase contrast by 80%
contrasted_image_arr = np.clip((image_arr - mean) * contrast_factor + mean, 0, 255).astype(np.uint8)

contrasted_image = Image.fromarray(contrasted_image_arr)
contrasted_image

### Filtering colours

We can turn off some colour channels and leave the rest on. For instance, we could just leave the red channel on.

In [ ]:
red_channel_image_arr = image_arr.copy()  # shallow copy of the array to prevent changes from modifying the original array
red_channel_image_arr[:, :, 1] = 0  # set green channel to 0
red_channel_image_arr[:, :, 2] = 0  # set blue channel to 0

red_channel_image = Image.fromarray(red_channel_image_arr)
red_channel_image

### Rotating an image

We can rotate an image by using the `np.rot90()` function

In [ ]:
acw_rotated_image_arr = np.rot90(image_arr, k = 3)  # k = 1 represents anti-clockwise rotation

acw_rotated_image = Image.fromarray(acw_rotated_image_arr)
acw_rotated_image

In [ ]:
cw_rotated_image_arr = np.rot90(image_arr, k = -1)  # k = -1 represents clockwise rotation

cw_rotated_image = Image.fromarray(cw_rotated_image_arr)
cw_rotated_image

### Flipping an image

We can flip an image about the horizontal or vertical axis by using the `np.flip()` function

In [ ]:
# Horizontal flip
h_flipped_image_arr = np.flip(image_arr, axis = 1)

h_flipped_image = Image.fromarray(h_flipped_image_arr)
h_flipped_image

In [ ]:
# Vertical flip
v_flipped_image_arr = np.flip(image_arr, axis = 0)

v_flipped_image = Image.fromarray(v_flipped_image_arr)
v_flipped_image

### Kernels or filters
Kernel-based or filter-based methods are a set of methods in image processing that use the available pixel value data along with the relative positions to perform various tasks such as blurring, edge detection, and so on.

We can think of kernels as boxes of a limited size, say `kernel_size`, moving across the image, and performing these operations.

Say we want to achieve blurring. The box must have a well-defined center pixel in this case because we want to substitute new values for the pixels, thus the kernel size should be an odd number.

We need to avoid the box leaving the bounds of the image. As the box centers itself on a pixel, we must ensure it starts iterating from `kernel_size // 2` after the first pixel and the same value before the last pixel, on both, the height and width dimensions.

Let's look at this using our greyscale version of the image

Returning to our image:

In [ ]:
kernel_size = 5  # must be odd as it centers on a pixel
hk = kernel_size // 2  # 'half kernel'; this helps in determining the neighborhood range

# Create an empty array of the same shape as the original to store the blurred image
blurred_image_arr = np.zeros(greyscale_image_arr.shape)

# Apply a box blur to the central area of the image (i.e., minus the edges)
for i in range(hk, greyscale_image_arr.shape[0] - hk):  # iterating over image height
    for j in range(hk, greyscale_image_arr.shape[1] - hk):  # iterating over image width

        # Extract the neighborhood defined by the kernel size
        neighborhood = greyscale_image_arr[
            i - hk : i + hk + 1,  # +1 to account for the fact that the end index is exclusive
            j - hk : j + hk + 1
        ]

        # Calculate the mean of the neighborhood for each color channel
        blurred_image_arr[i, j] = np.mean(neighborhood)


blurred_image_arr = blurred_image_arr.astype(np.uint8)  # as Image.fromarray() cannot handle float values

# Convert the output array back to an image and display it
blurred_image = Image.fromarray(blurred_image_arr)
blurred_image